# 直接使用bos进行sft并进行评估

<span style="font-size: 20px; font-weight: bold;">注意：您使用该案例默认的数据和模型训练时，会产生一定费用。计费方式参考：https://cloud.baidu.com/doc/WENXINWORKSHOP/s/6lrk4bgxb</span>

千帆Python SDK 现已支持直接使用bos进行训练，无需通过千帆平台中转。

本文将基于qianfan==0.3.0展示通过bos进行sft训练，并使用微调后的模型进行评估。

In [ ]:
! pip install "qianfan>=0.3.0" -U

## 前置准备
- 初始化千帆安全认证AK、SK

In [13]:
import os 

os.environ["QIANFAN_ACCESS_KEY"] = "your_ak"
os.environ["QIANFAN_SECRET_KEY"] = "your_sk"

## 数据集加载

千帆SDK提供了数据集实现帮助我们可以快速的加载本地的数据集到内存，并通过设定DataSource数据源以保存至本地\千帆平台\bos。

从本地数据集上传到BOS

In [ ]:
from qianfan.dataset import Dataset
from qianfan.dataset.data_source import BosDataSource
from qianfan.dataset.data_source.base import FormatType

bos_bucket_name = "your_bucket_name"
bos_bucket_file_path = "test"

bds = BosDataSource(region="bj", bucket=bos_bucket_name, bos_file_path=f"/{bos_bucket_file_path}/train_data.jsonl")

ds = Dataset.load(data_file="./data/fin_cqa_train.jsonl")
ds.save(bds, should_overwrite_existed_file=True)

### LLMFinetune 训练
`LLMFinetune` 实现了SFT逻辑的trainer，它内部组装了SFT所需要的基本`Pipeline`, 用于串联数据->训练->模型发布->服务调用等步骤

目前Dataset支持从bos读取数据，因此可以很方便的进行finetune。

In [21]:
from qianfan.trainer import LLMFinetune

# 首先需要先加载测试数据集，这里以加载bos上的数据集为例子：
# 使用bos训练有两种方法，一种是传入bos数据集，一种是直接传入bos路径，这里我们使用bos数据集为例

bos_ds = Dataset.load(bds,file_format=FormatType.Jsonl)

trainer = LLMFinetune(
    train_type="ERNIE-Speed-8K",
    # dataset_bos_path=f"bos:/{bos_bucket_name}/{bos_bucket_file_path}",
    dataset=bos_ds
)

[INFO] [02-01 17:59:41] bos.py:229 [t:139840090355520]: cache was outdated, start to update bos cache
[INFO] [02-01 17:59:42] bos.py:243 [t:139840090355520]: ready to fetch a file from bos path: /data/train_data.jsonl in bucket sdk-test


### 运行任务
同步运行trainer，训练直到模型发布完成

In [23]:
trainer.run()
print(trainer.result)

[INFO] [02-01 17:59:51] actions.py:468 [t:139840090355520]: [train_action] fine-tune running... task_name:turbo_0725_S3GYG current status: RUNNING, 1% check train task log in https://console.bce.baidu.com/qianfan/train/sft/job-cyihskbbi7kg/task-qq8pj4txzjvq/detail/traininglog
[INFO] [02-01 18:00:22] actions.py:468 [t:139840090355520]: [train_action] fine-tune running... task_name:turbo_0725_S3GYG current status: RUNNING, 1% check train task log in https://console.bce.baidu.com/qianfan/train/sft/job-cyihskbbi7kg/task-qq8pj4txzjvq/detail/traininglog
[INFO] [02-01 18:00:52] actions.py:468 [t:139840090355520]: [train_action] fine-tune running... task_name:turbo_0725_S3GYG current status: RUNNING, 3% check train task log in https://console.bce.baidu.com/qianfan/train/sft/job-cyihskbbi7kg/task-qq8pj4txzjvq/detail/traininglog
[INFO] [02-01 18:01:23] actions.py:468 [t:139840090355520]: [train_action] fine-tune running... task_name:turbo_0725_S3GYG current status: RUNNING, 3% check train task l

获取finetune任务输出：

In [18]:
trainer.result

[{'task_id': 19061,
  'job_id': 11630,
  'model_id': 'am-p02p43it6pqt',
  'model_version_id': 'amv-vge90genq6ed',
  'model': <qianfan.model.model.Model at 0x7fe0ea0f95e0>}]

从训练结果中获取模型对象

In [2]:
from qianfan.model import Model

m = trainer.result[0]['model']
m.auto_complete_info()

在完成模型的微调后，我们可以使用它来实现千帆平台评估任务：

首先获取/创建千帆平台待评估的测试集：

In [3]:
import os
from qianfan.dataset import Dataset
from qianfan.dataset import (
    DataTemplateType,
    DataStorageType,
)

qianfan_dataset_name = "random_fin_cqa_test"

qianfan_ds = Dataset.load(data_file="./data/fin_cqa_test.jsonl")

qianfan_ds = qianfan_ds.save(
    qianfan_dataset_create_args={
        "name": qianfan_dataset_name,
        "template_type": DataTemplateType.NonSortedConversation,
        "storage_type": DataStorageType.PrivateBos,
        "storage_id": bos_bucket_name,
        "storage_path": f'/{bos_bucket_file_path}',
    },
    does_release=True,
)

True

接着实例化评估器，分别是
+ `QianfanRefereeEvaluator`: 千帆平台的裁判员评估器，使用大模型对被评估大模型的回答打分，可以自定打分 prompt，打分步骤等信息，需要app_id
+ `QianfanRuleEvaluator`: 千帆平台的规则评估器，会根据评估结果计算出一系列统计指标


In [5]:
from qianfan.evaluation.evaluator import QianfanRefereeEvaluator, QianfanRuleEvaluator
from qianfan.evaluation.consts import QianfanRefereeEvaluatorDefaultMetrics, QianfanRefereeEvaluatorDefaultSteps, QianfanRefereeEvaluatorDefaultMaxScore

your_app_id = 1 # replace it with your qianfan app id

qianfan_evaluators = [
    QianfanRefereeEvaluator(
        app_id=your_app_id,
        prompt_metrics=QianfanRefereeEvaluatorDefaultMetrics,
        prompt_steps=QianfanRefereeEvaluatorDefaultSteps,
        prompt_max_score=QianfanRefereeEvaluatorDefaultMaxScore,
    ),
    QianfanRuleEvaluator(using_accuracy=True, using_similarity=True),
]

通过`EvaluationManager.eval`方法，我们可以将需要评估的`Model`以及`Dataset`传入，即可发起评估：

In [6]:
from qianfan.evaluation import EvaluationManager

em = EvaluationManager(qianfan_evaluators=qianfan_evaluators)
result = em.eval([m], qianfan_ds)

In [7]:
print(result.result_dataset.list())

[{'模型名称': 'm_19061_11630', '模型版本': 'V1', 'Prompt': '下文中其他自然灾害导致需求减少事件对应的原因涉及的地区是？风险因素:国内外因天气等自然灾害导致用肥量锐减', 'Completion（模型回答）': '很抱歉，文中没有提及下文中其他自然灾害导致需求减少事件对应的原因涉及的地区是。', 'Response（参考答案）': '国内外', 'BLEU-4': '0.00%', 'ROUGE-1': '0.00%', 'ROUGE-2': '0.00%', 'ROUGE-L': '0.00%', '裁判员模型打分': '2', '裁判员模型打分理由': '回答者给出了一个“很抱歉，文中没有提及”的回答，这表明他们确实在寻找与问题相关的信息，但未能找到。然而，由于这个答案并未提供任何关于问题所需的信息，因此评分为2。'}, {'模型名称': 'm_19061_11630', '模型版本': 'V1', 'Prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的产品是？在国际奶粉价格下降压力下,国内奶价仍有下降空间', 'Completion（模型回答）': '- 产品：奶粉', 'Response（参考答案）': '奶', 'BLEU-4': '0.00%', 'ROUGE-1': '0.00%', 'ROUGE-2': '0.00%', 'ROUGE-L': '0.00%', '裁判员模型打分': '4', '裁判员模型打分理由': '回答的产品“奶粉”是正确的，因为问题中提到的“国际奶粉价格下降压力下”表明了涉及的产品是奶粉。同时，回答没有出现用词不当或与问题要求不符的情况，所以给出了4分的评分。'}, {'模型名称': 'm_19061_11630', '模型版本': 'V1', 'Prompt': '下文中市场价格下降导致市场价格下降事件对应的结果涉及的产品是？但由于7-ACA市场价格大幅下跌，跌幅超过50%，导致公司相关原料药销售价格随之下跌', 'Completion（模型回答）': '- 产品：7-ACA', 'Response（参考答案）': '原料药', 'BLEU-4': '0.00%', 'ROUGE-1': '0.00%', 'ROUGE-2': '0.00%', 'ROUGE-L